In [1]:
import pandas as pd
import numpy as np
from stop_words import get_stop_words

In [2]:
data = pd.read_csv('data_1/dataset.csv')
data

,text,udk
0,"Науч. и техн. бки, 2016, № 141УДК 001.89Т. В. ...",001.89
1,"Науч. и техн. бки, 2016, № 183ПРОБЛЕМЫ ИНФОРМА...",02(063)
2,"Науч. и техн. бки, 2016, № 156УДК 023М. И. Рас...",023
3,"Науч. и техн. бки, 2016, № 151УДК 025.44/.47Е....",025.44-.47
4,"Науч. и техн. бки, 2016, № 169УДК 026:574+027....",026-574+027.625
...,...,...
664,БИБЛИОТЕЧНОЕ И СПРАВОЧНОИНФОРМАЦИОННОЕ ОБСЛУ...,025.5
665,БИБЛИОТЕЧНАЯ ПРОФЕССИЯ. КАДРЫ. ОБРАЗОВАНИЕ У...,027.7
666,УДК 37:004+02:37 https://doi.org/10.33186/10...,37-004+02-37
667,УДК 655.552+02+002 https://doi.org/10.33186/...,655.552+02+002


## Text preprocessing

In [3]:
def replace_numbers(sentence: str) -> str:
    new_words =[]
    for word in sentence.split():
        if word.isalpha():
            new_words.append(word)
    return ' '.join(new_words)  
                                                    
def remove_stopwords(sentence: str) -> str:
    new_words = []
    for word in sentence.split():
        if word not in get_stop_words('russian') + get_stop_words('english'):
            new_words.append(word)
    return ' '.join(new_words)

In [4]:
def normalize(sentence: str) -> str:
    sentence = replace_numbers(sentence)
    sentence = remove_stopwords(sentence)
    return sentence

In [5]:
%%time
data['text'] = data['text'].apply(normalize)

Wall time: 17 s


## paraphrase-multilingual-MiniLM-L12-v2 Transformer
https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2

In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

In [8]:
model = SentenceTransformer(model_name)

In [9]:
def mlm_embeddings(sentence: str) -> np.ndarray:
    '''Энкодер берет на вход текст макс. длины 128. Если на вход подается текст большей длины, то оно обрезается.
    Функция получается текст произвольной длины, делит ее на куски по 128 слов. Куски объединяются в один список и подаются
    энкодеру. Каждый кусок (размером 128) преобразуется энкодером в 328 мерный вектор. 
    Функция возвращает '(len(sentence)//128)*328+328'- мерный np.ndarray объект.'''
    
    embeddings = np.array([])
    
    sentence = sentence.split()
    sentence_chunks = []

    start = 0
    window_size = 128
    total_len = len(sentence)

    loop = True
    while loop:
        end = start + window_size

        if end >= total_len:
            loop = False
            end = total_len

        sentence_chunks.append(' '.join(sentence[start:end]))    
        start = end
        
    outputs = model.encode(sentence_chunks)
    embeddings = np.append(embeddings, outputs.ravel())

    return embeddings

def same_length_embeddings(embeddings: np.ndarray) -> np.ndarray:
    '''Для оценки косинуса подобия, необходиы вектора одной размерности. В нашем случае вектора получаются разных размерностей.
    Функция приводит каждый вектор к длине максимального объекта, добавляением 0-лей в конце.'''
    return np.append(embeddings, [0] * (max_length - (embeddings.shape[0])))

In [10]:
%%time
data['mlm_embeddings'] = data['text'].apply(mlm_embeddings)
max_length = np.max(data['mlm_embeddings'].apply(len))
data['mlm_embeddings'] = data['mlm_embeddings'].apply(same_length_embeddings)

Wall time: 19min 33s


In [11]:
data

,text,udk,mlm_embeddings
0,ЕрёменкоРязанский государственный университет ...,001.89,"[-0.042462702840566635, 0.16254602372646332, -..."
1,ИНФОРМАЦИОННОГО ОБЩЕСТВАУДК ЗемсковГПНТБ Конфе...,02(063),"[-0.18341054022312164, 0.03176235407590866, -0..."
2,РассадинаВладимирский областной колледж культу...,023,"[0.14342175424098969, 0.07016211003065109, -0...."
3,ЗайцеваГПНТБ РоссииРазвитие классификацийбибли...,025.44-.47,"[-0.17331849038600922, 0.06164005771279335, 0...."
4,ЛещинскаяРоссийская государственная библиотека...,026-574+027.625,"[0.06895367056131363, 0.28679823875427246, 0.1..."
...,...,...,...
664,БИБЛИОТЕЧНОЕ И СПРАВОЧНОИНФОРМАЦИОННОЕ ОБСЛУЖИ...,025.5,"[0.0846431627869606, -0.028025314211845398, -0..."
665,БИБЛИОТЕЧНАЯ ОБРАЗОВАНИЕ УДК Комлева Новосибир...,027.7,"[0.058247748762369156, 0.24192902445793152, -0..."
666,УДК Мухаметшин Институт археологии Халикова Ак...,37-004+02-37,"[-0.02187773585319519, 0.14094169437885284, -0..."
667,УДК Зверевич ГПНТБ Российская Универсум Леонов...,655.552+02+002,"[-0.03286799415946007, 0.18403780460357666, -0..."


In [12]:
data['mlm_embeddings'][0].shape

(29568,)

## universal-sentence-encoder-multilingual Transformer
https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3

In [13]:
import tensorflow_hub as hub
import tensorflow_text

In [14]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [15]:
def use_embeddings(sentence: str) -> np.ndarray:
    '''Энкодер принимает тексты произвольной длины и возвращается 512 мерный TensorFlow объект.
    Функция преобразует TensorFlow эмбеддинг в np.ndarray объект.'''
    return np.concatenate(embed(sentence).numpy())

In [17]:
%%time
data['use_embeddings'] = data['text'].apply(use_embeddings)

Wall time: 3min 57s


In [19]:
data

,text,udk,mlm_embeddings,use_embeddings
0,ЕрёменкоРязанский государственный университет ...,001.89,"[-0.042462702840566635, 0.16254602372646332, -...","[-0.05500672, 0.05491702, 0.050894015, 0.00420..."
1,ИНФОРМАЦИОННОГО ОБЩЕСТВАУДК ЗемсковГПНТБ Конфе...,02(063),"[-0.18341054022312164, 0.03176235407590866, -0...","[-0.04918318, -0.030474897, -0.015696421, 0.00..."
2,РассадинаВладимирский областной колледж культу...,023,"[0.14342175424098969, 0.07016211003065109, -0....","[-0.055245362, 0.046793673, 0.050325207, -0.04..."
3,ЗайцеваГПНТБ РоссииРазвитие классификацийбибли...,025.44-.47,"[-0.17331849038600922, 0.06164005771279335, 0....","[-0.054188013, 0.03612344, -0.0127063105, 0.05..."
4,ЛещинскаяРоссийская государственная библиотека...,026-574+027.625,"[0.06895367056131363, 0.28679823875427246, 0.1...","[-0.05200468, 0.048524145, 0.022760656, -0.051..."
...,...,...,...,...
664,БИБЛИОТЕЧНОЕ И СПРАВОЧНОИНФОРМАЦИОННОЕ ОБСЛУЖИ...,025.5,"[0.0846431627869606, -0.028025314211845398, -0...","[-0.051134568, 0.04989279, 0.05091167, -0.0457..."
665,БИБЛИОТЕЧНАЯ ОБРАЗОВАНИЕ УДК Комлева Новосибир...,027.7,"[0.058247748762369156, 0.24192902445793152, -0...","[-0.038172144, 0.040904384, 0.049835395, -0.00..."
666,УДК Мухаметшин Институт археологии Халикова Ак...,37-004+02-37,"[-0.02187773585319519, 0.14094169437885284, -0...","[-0.052403744, 0.052356265, -0.012128776, 0.02..."
667,УДК Зверевич ГПНТБ Российская Универсум Леонов...,655.552+02+002,"[-0.03286799415946007, 0.18403780460357666, -0...","[-0.048506793, -0.039012127, 0.0485149, -0.048..."


In [20]:
data['use_embeddings'][0].shape

(512,)

### Cosine similarity comparison

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
%%time
ID = 2 # id текста для сравнения семантического подобия
similarity_df = pd.DataFrame({'udk': data['udk'].tolist()})
similarity_df['mlm_similarity'] = cosine_similarity([data['mlm_embeddings'][ID]], data['mlm_embeddings'].tolist())[0]
similarity_df['use_similarity'] = cosine_similarity([data['use_embeddings'][ID]], data['use_embeddings'].tolist())[0]

In [36]:
similarity_df.sort_values('mlm_similarity', ascending=False)

,udk,mlm_similarity,use_similarity
2,023,1.000000,1.000000
23,023.1,0.612710,0.723239
131,023-37,0.581146,0.728810
312,027.7,0.579690,0.536760
198,02-37,0.570535,0.705393
...,...,...,...
113,002.2,0.172787,0.438249
92,004.75,0.168685,0.318017
449,37-004+001-004+008-004,0.152515,0.372465
343,001.102+026.06,0.144540,0.438024


In [37]:
similarity_df.sort_values('use_similarity', ascending=False)

,udk,mlm_similarity,use_similarity
2,023,1.000000,1.000000
130,023(09),0.551831,0.773009
170,02-378,0.564615,0.769916
451,02-378,0.446922,0.764613
316,02-378,0.506207,0.759278
...,...,...,...
152,026.06,0.308524,0.220514
643,026.06,0.390266,0.219460
487,001.816,0.375031,0.217133
567,02-004,0.208389,0.215070


In [27]:
data.to_csv('data_embeddings.csv', index=False)